In [43]:
from os import walk
from os.path import join
from pathlib import Path

# path = Path(Path.cwd()) / "pipelines/reports_disque_denuncia/data/partition_directory/ano_particao=2024/mes_particao=07/data_particao=2024-07-01"
path = Path("pipelines/reports_disque_denuncia/data/partition_directory")
data_type = 'csv'
found = False

In [41]:
if path.is_dir():
    print('dir')
else:
    print('file')
    print(path.parent)
    path = path.parent

file
\pipelines\reports_disque_denuncia\data


In [42]:
for subdir, _, filenames in walk(str(path)):
    for fname in filenames:
        if fname.endswith(f".{data_type}"):
            file = join(subdir, fname)
            print(f"Found {data_type.upper()} file: {file}")
            found = True
            break
    if found:
        break

In [44]:
import pandas as pd
from typing import List

In [45]:
def to_partitions(
    data: pd.DataFrame,
    partition_columns: List[str],
    savepath: str,
    data_type: str = "csv",
    suffix: str = None,
    build_json_dataframe: bool = False,
    dataframe_key_column: str = None,
) -> List[Path]:  # sourcery skip: raise-specific-error
    """Save data in to hive patitions schema, given a dataframe and a list of partition columns.
    Args:
        data (pandas.core.frame.DataFrame): Dataframe to be partitioned.
        partition_columns (list): List of columns to be used as partitions.
        savepath (str, pathlib.PosixPath): folder path to save the partitions
    Exemple:
        data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }
        to_partitions(
            data=pd.DataFrame(data),
            partition_columns=['ano','mes','sigla_uf'],
            savepath='partitions/'
        )
    """
    saved_files = []
    if isinstance(data, (pd.core.frame.DataFrame)):
        savepath = Path(savepath)

        # create unique combinations between partition columns
        unique_combinations = (
            data[partition_columns]
            .drop_duplicates(subset=partition_columns)
            .to_dict(orient="records")
        )

        for filter_combination in unique_combinations:
            patitions_values = [
                f"{partition}={value}" for partition, value in filter_combination.items()
            ]

            # get filtered data
            df_filter = data.loc[
                data[filter_combination.keys()].isin(filter_combination.values()).all(axis=1),
                :,
            ]
            df_filter = df_filter.drop(columns=partition_columns).reset_index(drop=True)

            # create folder tree
            filter_save_path = Path(savepath / "/".join(patitions_values))
            filter_save_path.mkdir(parents=True, exist_ok=True)
            if suffix is not None:
                file_filter_save_path = Path(filter_save_path) / f"data_{suffix}.{data_type}"
            else:
                file_filter_save_path = Path(filter_save_path) / f"data.{data_type}"

            if build_json_dataframe:
                df_filter = to_json_dataframe(df_filter, key_column=dataframe_key_column)

            if data_type == "csv":
                # append data to csv
                df_filter.to_csv(
                    file_filter_save_path,
                    index=False,
                    mode="a",
                    header=not file_filter_save_path.exists(),
                )
                saved_files.append(file_filter_save_path)
            elif data_type == "parquet":
                dataframe_to_parquet(dataframe=df_filter, path=file_filter_save_path)
                saved_files.append(file_filter_save_path)
            else:
                raise ValueError(f"Invalid data type: {data_type}")
    else:
        raise BaseException("Data need to be a pandas DataFrame")

    return saved_files

In [46]:
data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }


In [50]:
def to_partitions(
    data: pd.DataFrame,
    partition_columns: List[str],
    savepath: str,
    data_type: str = "csv",
    suffix: str = None,
    build_json_dataframe: bool = False,
    dataframe_key_column: str = None,
) -> List[Path]:  # sourcery skip: raise-specific-error
    """Save data in to hive patitions schema, given a dataframe and a list of partition columns.
    Args:
        data (pandas.core.frame.DataFrame): Dataframe to be partitioned.
        partition_columns (list): List of columns to be used as partitions.
        savepath (str, pathlib.PosixPath): folder path to save the partitions
    Exemple:
        data = {
            "ano": [2020, 2021, 2020, 2021, 2020, 2021, 2021,2025],
            "mes": [1, 2, 3, 4, 5, 6, 6,9],
            "sigla_uf": ["SP", "SP", "RJ", "RJ", "PR", "PR", "PR","PR"],
            "dado": ["a", "b", "c", "d", "e", "f", "g",'h'],
        }
        to_partitions(
            data=pd.DataFrame(data),
            partition_columns=['ano','mes','sigla_uf'],
            savepath='partitions/'
        )
    """
    saved_files = []
    if isinstance(data, (pd.core.frame.DataFrame)):
        savepath = Path(savepath)

        # create unique combinations between partition columns
        unique_combinations = (
            data[partition_columns]
            .drop_duplicates(subset=partition_columns)
            .to_dict(orient="records")
        )

        for filter_combination in unique_combinations:
            patitions_values = [
                f"{partition}={value}" for partition, value in filter_combination.items()
            ]

            # get filtered data
            df_filter = data.loc[
                data[filter_combination.keys()].isin(filter_combination.values()).all(axis=1),
                :,
            ]
            df_filter = df_filter.drop(columns=partition_columns).reset_index(drop=True)

            # create folder tree
            filter_save_path = Path(savepath / "/".join(patitions_values))
            filter_save_path.mkdir(parents=True, exist_ok=True)
            if suffix is not None:
                file_filter_save_path = Path(filter_save_path) / f"data_{suffix}.{data_type}"
            else:
                file_filter_save_path = Path(filter_save_path) / f"data.{data_type}"

            if build_json_dataframe:
                df_filter = to_json_dataframe(df_filter, key_column=dataframe_key_column)

            if data_type == "csv":
                # append data to csv
                df_filter.to_csv(
                    file_filter_save_path,
                    index=False,
                    mode="a",
                    header=not file_filter_save_path.exists(),
                )
                saved_files.append(file_filter_save_path)
            elif data_type == "parquet":
                dataframe_to_parquet(dataframe=df_filter, path=file_filter_save_path)
                saved_files.append(file_filter_save_path)
            else:
                raise ValueError(f"Invalid data type: {data_type}")
    else:
        raise BaseException("Data need to be a pandas DataFrame")

    return saved_files

In [51]:

to_partitions(
    data=pd.DataFrame(data),
    partition_columns=['ano','mes','sigla_uf'],
    savepath='tmp/pipelines/reports_disque_denuncia/data/partition_directory/'
)

[WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2020/mes=1/sigla_uf=SP/data.csv'),
 WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2021/mes=2/sigla_uf=SP/data.csv'),
 WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2020/mes=3/sigla_uf=RJ/data.csv'),
 WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2021/mes=4/sigla_uf=RJ/data.csv'),
 WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2020/mes=5/sigla_uf=PR/data.csv'),
 WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2021/mes=6/sigla_uf=PR/data.csv'),
 WindowsPath('tmp/pipelines/reports_disque_denuncia/data/partition_directory/ano=2025/mes=9/sigla_uf=PR/data.csv')]